In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
import os
import subprocess
import shapely as shp
from datetime import datetime

# make sure we are at the top of the repo
wd = subprocess.check_output('git rev-parse --show-toplevel', shell = True)
os.chdir(wd[:-1]) #-1 removes \n

In [2]:
# read in data

df = gp.read_file('data/citi_docks/citi_docks.shp')

subway = pd.read_csv('data/subway.csv')

In [3]:
# combine lat and lon to one column
# Create Point Geometry for based on lonlat column
subway['lonlat']=zip(subway['Station Longitude'], subway['Station Latitude'])
subway['geometry']=subway[['lonlat']].applymap(lambda x:shp.geometry.Point(x))
subway = gp.GeoDataFrame(subway[['geometry']], geometry = 'geometry')

# handle coordinate system
subway.crs = {'init' :'epsg:4326'}
subway = subway.to_crs(epsg=2263)
subway.head()

,geometry
0,POINT (984779.6435399343 179876.8295590221)
1,POINT (984779.6435399343 179876.8295590221)
2,POINT (983265.2682330192 177963.0294655868)
3,POINT (983265.2682330192 177963.0294655868)
4,POINT (983265.2682330192 177963.0294655868)


In [4]:
subway.distance(df.geometry[0]).min()

2789.4372109760661

In [5]:
# are citibikes near subway stations? near = within a quarter mile
a = []
b = []
df['subway_ct'] = 0
for i in range(len(df)):
    # includes stations not in Manhattan, return 0 for CB. 
    try: 
        a.append(len(subway[subway.geometry.intersects(df.iloc[i].geometry.buffer(5280/4))]))
        b.append(subway.distance(df.geometry[i]).min() / 5280)
    except: 
        a.append(0)
    print '\r',"%",str((i+1)*100./len(df))[:4],
df['subway_ct'] = a
df['subway_dist'] = b
df = df.reindex()

% 100.


In [6]:
# export final data
if not os.path.exists('data/citi_docks2/'):
    os.makedirs('data/citi_docks2/')
    
df.to_file('data/citi_docks2/citi_docks2.shp')